In [6]:
# virtualenv 하는 법 [ 리눅스 ]
# pip3 install virtualenv
# virtualenv [name of virtualenv]
# virtualenv [name of virtualenv] --python=3.6
# source [name of virtualenv]/bin/activate
#deactivate

In [1]:
# mps = 맥북의 gpu 가속 모드 
import torch

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [ ]:
# cuda = nvidia GPU 가속 모드
# developer.nvidia.com/cuda-toolkit-archive => GPU와 파이썬 버전 확인 필요
# cuDNN도 설치 필요 

In [ ]:
# [ 기본연산 ]
# (scalar)스칼라 => 하나의 값
# (tensor)벡터 => 두개 이상의 수치로 표현한 값
    # torch.add/sub/mul/div/dot => 사칙 연산 함수
# (tensor)행렬 => 두개 이상의 벡터값으로 구성된 값 / "연산 속도를 빠르게 진행할 수 있는 선형 대수의 기본 단위"
    # matmul : 행렬곱
# (tensor)텐서 => 2차원 이상의 배열  / 이거 곱 계산은 두가지 ( 그냥곱 / 행렬곱 )


In [3]:
import torch
if torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    print("mps")
else:
    DEVICE = torch.device('cpu')
    print("cpu")

mps


In [4]:
BATCH_SIZE = 64
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 10

In [6]:
x = torch.randn(BATCH_SIZE, INPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)

y = torch.randn(BATCH_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)

w1 = torch.randn(INPUT_SIZE, HIDDEN_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)

w2 = torch.randn(HIDDEN_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)


In [11]:
# 데이터와 가중치 x, y, w1, w2 가 정의되어 있다는 가정 하에

learning_rate = 1e-6
for t in range(1, 501):
    # Forward Pass : 예측값 계산
    # mm : 행렬곱, clamp(min = 0)은 ReLU 활성화 함수 
    y_pred = x.mm(w1).clamp(min = 0).mm(w2) # 이건 예측값
    
    # Loss 계산 (MSE와 유사한 형태))
    loss = (y_pred - y).pow(2).sum()
    
    if t %100 ==0:
        print("Iteration : ", t ,"\t", "Loss: ", loss.item())
        
    # Backward Pass : Autograd가 미분(기울기) 계산
    loss.backward()
    
    # 4. 가중치 업데이트( 기록 안남기려고 no_grad 사용)
    with torch.no_grad( ):
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 기울기 초기화
        w1.grad.zero_()
        w2.grad.zero_()

Iteration :  100 	 Loss:  534.28759765625
Iteration :  200 	 Loss:  3.391860008239746
Iteration :  300 	 Loss:  0.040327977389097214
Iteration :  400 	 Loss:  0.0008995939861051738
Iteration :  500 	 Loss:  0.00010313434177078307


## Autograd vs gridsearchCV ( : 파라미터 관련이라)
    - 파라미터 vs 하이퍼파라미터
        => 파라미터 : 가중치 , 편향
        => 하이퍼파라미터 ㅣ 학습률, 배치 크기, 층 수 
    - gridsearch : 하이퍼 파라미터, 사용자가 미리 정해준 것 대입 후 가장 성과가 좋은 것 적용
    - Autograd : 파라미터, 어떤 가중치를 고려해야 정답에 가까워질지 손실실함수를 미분하여 기울기 구해서 제안 -> optimizer 통해서 

## Batch_Size : 파라미터를 업데이트 할 때, 계산되는 데이터의 개수
## Hidden_size : 결과에 한번 더 계산되는 파라미터 수 

## gradient : 기울기와 방향 
## Relu 함수 : x에 w1층 곱하고 렐르함수 적용, 이후 w2 다시 곱함 ( 이 라인에서는 행렬곱 )
## 이후 오차.pow(2).sum()로 제곱합을 얻어냄
## 그 다음, bakward()로 gradient 계산

## 후 해당 시점의 gradient 값 고정

## w1.grad.lr - w1
## w2.grad.lr - w2 이러한 가중치 변화는 덮어쓰기화 됨

## 결국, 한번에 루프에서 일어나는 것 
    # 가중치는 계속해서 덮어쓰기 됨
    # gradient는 기존 수치가 반영되기 때문에 초기화해서 새롭게 구해진 가중치로 구한 새로운 gradient를 구해서 다시 가중치 조정이 이뤄지는 것임

